<a href="https://colab.research.google.com/github/tlsgptj/MLP/blob/MLP2/%EC%A0%84%EB%A0%A5%EB%9F%89(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("전력 revenge2.csv", encoding="cp949")

In [ ]:
df

In [ ]:
df = df.drop(columns = ["Unnamed: 8", "Unnamed: 9"])
#NaN 데이터는 제거해주자

In [ ]:
df

In [ ]:
df.head(20)

In [ ]:
df.tail(20)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.info()
#문자형 데이터가 없어서 다행인듯

In [ ]:
#전력량 정규화가 필요
df["month_normalization"] = df["monty"].astype(float)/abs(df["monty"].astype(float).max())
df[["monty", "month_normalization"]].head()

In [ ]:
df["MWh*1000"] = df["MWh*1000"].astype(float)/abs(df["MWh*1000"].astype(float).max())
df["MWh*1000"].head()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
plt.scatter(df["bs_high temp"], df["MWh_normalization"])
plt.xlabel("bs_high temp")
plt.ylabel("MWh_normalization")
plt.title("부산 기온에 따른 전력사용량")
plt.show()

In [ ]:
plt.scatter(df["gj_high temp"], df["MWh_normalization"])
plt.xlabel("gj_high temp")
plt.ylabel("MWh_normalization")
plt.title("광주 기온에 따른 전력사용량")
plt.show()

In [ ]:
plt.scatter(df["s_high temp"], df["MWh_normalization"])
plt.xlabel("s_high temp")
plt.ylabel("MWh_normalization")
plt.title("서울 기온에 따른 전력사용량")
plt.show()
#서울은 윗지방인지 전력사용량이 기온이 -10일때까지 분포하고 있었음

In [ ]:
df["bs_high temp"] = df["bs_high temp"].astype(float)/abs(df["bs_high temp"].astype(float).max())
df[["bs_high temp", "bs_high temp"]].head()

In [ ]:
df["s_high temp"] = df["s_high temp"].astype(float)/abs(df["s_high temp"].astype(float).max())
df[["s_high temp", "s_high temp"]].head()

In [ ]:
df["year"] = df["year"].astype(float)/abs(df["year"].astype(float).max())
df["year"].head()

In [ ]:
#MLP를 이용하여 부산의 전력량 예측해보기, 1차원데이터이므로 MLP 옵티마이저 아담을 사용하여 분석해본다
data = df["bs_high temp"]
target = df["MWh_normalization"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.009828583337366581
#1/1 [==============================] - 0s 92ms/step
#Next value prediction: [[0.82345855]]

In [ ]:
#이번에는 광주의 기온과 전력량을 예측해본다.
data = df["gj_high temp"]
target = df["MWh_normalization"]

In [ ]:
df_2 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
df_2

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='SGD', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#adam
#Test loss: 0.00728259515017271
#1/1 [==============================] - 0s 56ms/step
#Next value prediction: [[0.86865175]]

#SGD
#Test loss: 0.010611072182655334
#1/1 [==============================] - 0s 56ms/step
#Next value prediction: [[0.78998226]]

In [ ]:
#서울의 기온에 따른 전력량을 에측해보자
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
df_3

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#RMSprop
#Test loss: 0.005654627922922373
#1/1 [==============================] - 0s 84ms/step
#Next value prediction: [[0.8333498]]

In [ ]:
#광주 일별 광주 기온 예측을 해보겠다.
data = df["day"]
target = df["gj_high temp"]

In [ ]:
df_4 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
df_4

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adagrad', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adagrad
#Test loss: 0.033506181091070175
#1/1 [==============================] - 0s 55ms/step
#Next value prediction: [[0.51760966]]

In [ ]:
#서울시의 기온 예측
data = df["day"]
target = df["s_high temp"]

In [ ]:
df_5 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
df_5

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adadelta', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adadelta
#Test loss: 0.30417826771736145
#1/1 [==============================] - 0s 62ms/step
#Next value prediction: [[0.17683095]]

In [ ]:
#부산광역시의 기온예측
data = df["day"]
target = df["bs_high temp"]

In [ ]:
df_6 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
df_6

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adamax
#Test loss: 0.007760117296129465
#1/1 [==============================] - 0s 59ms/step
#Next value prediction: [[0.82375294]]

In [ ]:
#분석내용 표로 도출
data = {
    '무엇을 예측했는지': ['부산기온/전력', '기온/광주전력', '기온/서울전력', '일별/광주기온', '일별/서울기온', '일별/부산기온'],
    'target-data': ['MWh*1000', 'MWh*1000', 'MWh*1000', 'gj_high temp', 's_high temp', 'bs_high temp'],
    'Optimizer': ['adam', 'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adamax'],
    '오차': [0.009828583337366581, 0.010611072182655334, 0.005654627922922373, 0.033506181091070175, 0.30417826771736145, 0.007760117296129465],
    'Epoch': [150, 150, 150, 150, 150, 150],
    'Next value prediction': [0.82345855, 0.78998226, 0.8333498, 0.51760966, 0.17683095, 0.82375294]
}

df_7 = pd.DataFrame(data)

In [ ]:
df_7

In [ ]:
#각각의 옵티마이저를 사용했을 때 가장 오차가 적은 것을 찾아 이번엔 Epoch와 batch_size 등 각각의 것들을 비교해 오차가 작은 것을 선별할 예정이다.
import numpy as np
import matplotlib.pyplot as plt
x = df_7["Optimizer"]
y1 = df_7["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

In [ ]:
#전력량 MLP 데이터를 활용하여 다양한 MLP를 이용해 오차가 최소인 것을 결과로 도출해보았다.
#오차는 RMSprop 옵티마이저를 사용했을 때 가장 작았고, Adadelta를 사용했을 시 최대였다.
#예측값은 정규화된 값으로 보여짐

In [ ]:
#RMSprop를 이용하여 더 정교하게 오차가 최소화되는 조건을 찾아봄

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
   # keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    #keras.layers.Dense(48, activation='relu'),
#Test loss: 0.007946128025650978
#1/1 [==============================] - 0s 68ms/step
#Next value prediction: [[0.8650393]]

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.009651596657931805
#1/1 [==============================] - 0s 61ms/step
#Next value prediction: [[0.8439958]]

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_3['data'].iloc[i:i+window_size].values)
    y.append(df_3['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_3['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
model.save('mlp_model.h5')

In [ ]:
#keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
 #   keras.layers.Dense(24,
#epochs=500, batch_size=64)
#Test loss: 0.007640763185918331
#1/1 [==============================] - 0s 60ms/step
#Next value prediction: [[0.878584]]

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=500, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
 #keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
  #  keras.layers.Dense(48, activation='relu')
#epochs=500, batch_size=32)
#Test loss: 0.013502337038516998
#1/1 [==============================] - 0s 89ms/step
#Next value prediction: [[0.8500004]]

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
 #   keras.layers.Dense(24,
#epochs=1000, batch_size=64)
#Test loss: 0.008411530405282974
#1/1 [==============================] - 0s 63ms/step
#Next value prediction: [[0.8621399]]

In [ ]:
data = df["s_high temp"]
target = df["MWh*1000"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#(96, activation='relu', input_shape=(window_size,)),
 #   keras.layers.Dense(48,
#epochs=1000, batch_size=32)
#Test loss: 0.01408625952899456
#1/1 [==============================] - 0s 57ms/step
#Next value prediction: [[0.75207967]]

In [ ]:
#표로 정리
data = {
    "번호": [1, 2, 3, 4, 5, 6],
    'Keras의 Dense 층': ['48/24', '48/24', '96/48', '48/24', '96/48', '96/48'],
    '오차': [0.008262787945568562, 0.009651596657931805, 0.007946128025650978, 0.007640763185918331, 0.013502337038516998, 0.01408625952899456],
    'Epoch': [200, 200, 500, 500, 1000, 1000],
    'Next value prediction': [0.84887606, 0.8439958, 0.8650393, 0.878584, 0.8500004, 0.75207967],
    'Batch_size': [64, 32, 64, 64, 32, 32]
}

df_8 = pd.DataFrame(data, dtype=float)

In [ ]:
df_8

In [ ]:
#가장 오차가 작은 것을 알아보기 위해 그래프로 나타내보았다.
import numpy as np
import matplotlib.pyplot as plt
x = df_8["번호"]
y1 = df_8["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

#3	48/24	0.007641	500.0	0.878584	64.0
#이 모델이 가장 오차가 작게 나왔다.

#더 많은 비교를 해보고 싶었지만 대표적으로 많이 쓰는 48/24, 32/16, 96/48일 때의 조건을 기준으로 비교해보았다.

In [ ]:
pip install streamlit

In [ ]:
#모델의 결과를 가지고 streamlit을 활용하여 전력량을 예측해주는 프로토타입을 만들었다.
#이것으로 streamlit으로 구축해보겠다.
import streamlit as st
import tensorflow as tf
from tensorflow import keras

# MLP 모델 불러오기
model = keras.models.load_model('mlp_model.h5')

# Streamlit 앱 설정
st.title("서울시의 전력을 예측해보자!")

# 풍속 입력 필드
speed = st.number_input("MWh*1000", min_value=0.0)

# 예측 함수
def predict_power(electricity):
    power = model.predict([[electricity]])[0][0]
    return power

# 예측 버튼
if st.button("예측"):
    result = predict_power(speed)
    st.write(f"서울시 전기 소비량 예측: {result:.2f} MWh")

In [ ]:
pip install pyngrok

In [ ]:
!nohup streamlit run app.py --server url &

In [ ]:
url = ngrok.connect(port='80')
url